In [9]:
import os
import numpy as np
import random

import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Tuple, Dict, MultiDiscrete, MultiBinary

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy


In [16]:

class ShowerEnv(gym.Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60

    def step(self, action):
        self.state += action -1
        self.shower_length -= 1

        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1

        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [17]:
env = ShowerEnv()

In [22]:
env.observation_space.sample()

array([1.4877111], dtype=float32)

In [27]:
env.action_space.sample()

2

In [33]:
episodes = 500
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode: {episode}, Score: {score}')

Episode: 1, Score: -42
Episode: 2, Score: -28
Episode: 3, Score: 30
Episode: 4, Score: -60
Episode: 5, Score: -6
Episode: 6, Score: 6
Episode: 7, Score: -58
Episode: 8, Score: -6
Episode: 9, Score: -38
Episode: 10, Score: -60
Episode: 11, Score: -26
Episode: 12, Score: -60
Episode: 13, Score: -58
Episode: 14, Score: -8
Episode: 15, Score: -52
Episode: 16, Score: -46
Episode: 17, Score: -34
Episode: 18, Score: -2
Episode: 19, Score: -22
Episode: 20, Score: 8
Episode: 21, Score: -12
Episode: 22, Score: -18
Episode: 23, Score: 6
Episode: 24, Score: -22
Episode: 25, Score: -8
Episode: 26, Score: -58
Episode: 27, Score: -34
Episode: 28, Score: -8
Episode: 29, Score: -42
Episode: 30, Score: -32
Episode: 31, Score: -58
Episode: 32, Score: -6
Episode: 33, Score: -58
Episode: 34, Score: -30
Episode: 35, Score: -24
Episode: 36, Score: -60
Episode: 37, Score: -14
Episode: 38, Score: -56
Episode: 39, Score: 20
Episode: 40, Score: -32
Episode: 41, Score: -44
Episode: 42, Score: -42
Episode: 43, Sco